In [1]:
import pysam as ps
import numpy as np
import collections
import matplotlib.pyplot as plt
from numpy.core.defchararray import array, center
import pysam as ps
import numpy as np
from collections import defaultdict
from scipy.ndimage.measurements import label, standard_deviation
from scipy.stats.mstats_basic import kstest, normaltest
from sklearn.cluster import KMeans
import sys
from scipy.signal import savgol_filter
import math
from subprocess import call
import os.path
from utils import Gene, TSS, Point
from scipy import stats
from sklearn import svm
import sympy
import math
from math import e
import random
dic = {
    'gene':'./data/gene/GRCh37.gene.bed',
    'non_gene':'./data/gene/non_gene.bed',
    'non_gene_4':'./data/gene/non_gene_4.bed',
    'non_gene_2':'./data/gene/non_gene_2.bed',
    'fasta':'/home/jiay/Desktop/hg19/hg19.fa',
    'bam1':'./data/051.bam',
    'bam2':'./data/051_2.bam',
    'bam3':'./data/051_3.bam',
    'bam123':'./data/051_123.bam',
    'bam4':'./data/051_4.bam',
    'bam5':'./data/051_5.bam',
    'TSS_low':'./data/gene/low_expressed.bed',
    'TSS_HK':'./data/gene/HK.bed',
    'TSS_silent':'./data/gene/silent_gene_TSS.bed',
    'ATAC_hema_1':'./data/gene/ATAC_hema_1.bed',
    'ATAC_hema_2':'./data/gene/ATAC_hema_2.bed',
    'ATAC_Bcell':'./data/gene/ATAC_Bcell.bed'
    }
up = 1000
down = 1000
bamfile = ps.AlignmentFile(dic['bam123'],'rb')

In [2]:
TSS_pre = []
with open(dic['ATAC_hema_1'],'r') as f:
    for line in f:
        ll = line.strip().split('\t')
        TSS_pre.append(TSS(ll[0], int((int(ll[1])+int(ll[2]))/2)))
TSS_NonGene = []
with open(dic['non_gene_2'],'r') as f:
    for line in f:
        ll = line.strip().split('\t')
        TSS_NonGene.append(TSS(ll[0], int(ll[1])+int(1000)))
TSS_NonGene = TSS_NonGene[:len(TSS_pre)]

TSSes_x = TSS_pre + TSS_NonGene
TSSes_y = [1]*len(TSS_pre) + [0]*len(TSS_NonGene)
perm = random.sample(range(len(TSSes_x)),len(TSSes_x))
TSSes_x = np.array(TSSes_x)
TSSes_y = np.array(TSSes_y)
data_x = TSSes_x[perm[:len(perm)]]
data_y = TSSes_y[perm[:len(perm)]]

In [3]:
'''lstm_input'''

feature_matrix = []
for j, tss in enumerate(TSS_pre):
    chrom = tss.chrom
    start = tss.pos - up
    end = tss.pos + down
    up_end = np.zeros(up+down, dtype= int)
    down_end = np.zeros(up+down, dtype= int)
    long = np.zeros(up+down, dtype= int)
    short = np.zeros(up+down, dtype= int)
    cov = np.zeros(up+down, dtype= int)
    for r in bamfile.fetch(chrom[-1], start-500, end + 500):
        if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse:
            if r.reference_start + abs(r.isize) < start:
                continue
            if r.reference_start >= end:
                continue
            # ss = max(0, r.reference_start - start)
            # ee = min(r.reference_start - start + abs(r.isize), end - start)
            ss = r.reference_start - start
            ee = r.reference_start - start + abs(r.isize)
            if ss >= 0:
                up_end[ss] += 1
            else:
                ss = 0
            if ee < end - start:
                down_end[ee] += 1
            else:
                ee = end - start
            for i in range(ss, ee):
                cov[i] += 1
            if 200 >= abs(r.isize) > 130:
                for i in range(ss, ee):
                    long[i] += 1
            if abs(r.isize) <= 130:
                for i in range(ss, ee):
                    short[i] += 1
    k = 0
    win = 40
    feature_win = np.zeros((int((up+down)/win), 4), dtype= int)
    while k < (up+down)/win:
        ss = k * win
        ee = k * win + win
        ff = []
        ff.append(int(round(np.mean(cov[ss:ee]))))
        ff.append(int(round(np.mean(long[ss:ee]-short[ss:ee]))))
        ff.append(int(round(np.sum(abs(up_end[ss:ee]-down_end[ss:ee])))))
        wps = 0
        for r in bamfile.fetch(chrom[-1], ss+start, ee+start):
            if r.reference_start <= ss+start and r.reference_start+abs(r.isize) >= ee + start:
                wps += 1
            elif ee + start > r.reference_start > ss + start:
                wps -= 1
            elif ss+start < r.reference_start + abs(r.isize) < ee + start:
                wps -= 1
        ff.append(int(wps))
        feature_win[k] = np.array(ff)
        k =  k + 1
    feature_matrix.append(feature_win)
feature_matrix = np.array(feature_matrix)

lstm_x = []
for mat in feature_matrix:
    lstm_x.append(mat)
lstm_x = np.array(lstm_x)

In [4]:
'''cnn_input'''
raw_data = []
for j, tss in enumerate(TSS_pre):
    chrom = tss.chrom
    start = tss.pos - up
    end = tss.pos + down
#    distribution_matrix = np.zeros((int(up+down), 200), dtype=int)
    distribution_matrix = np.zeros((200,200),dtype=int)
    for r in bamfile.fetch(chrom[-1], start-500, end + 500):
        if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse and 50 < abs(r.isize) < 250:
            if r.reference_start + abs(r.isize) < start:
                continue
            if r.reference_start >= end:
                continue
            if r.reference_start < start:
                continue
            if r.reference_start + abs(r.isize) > end:
                continue
            ss = max(0, r.reference_start - start)
            relative_isize = abs(r.isize)-50
            distribution_matrix[ss//10,relative_isize] += 1
    raw_data.append(distribution_matrix)
raw_data = np.array(raw_data)

cnn_x = []
for mat in raw_data:
    cnn_x.append(mat)
cnn_x = np.array(cnn_x)

In [18]:
from keras.layers import Input, Embedding, LSTM, Dense, concatenate, Convolution1D, MaxPooling1D, Flatten, Bidirectional,Dropout
from keras.models import Model, load_model
from tensorflow import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
LL = keras.utils.to_categorical(data_y)

lstm_model = load_model('conv_lstm.h5')
trained_model = Model(inputs = lstm_model.input, outputs=lstm_model.get_layer('0').output)
trained_model.trainable = False
retrain_input = trained_model.predict([cnn_x,lstm_x])
retrain_input = np.array(retrain_input)

In [23]:
model = keras.Sequential()
model.add(Convolution1D(filters=100,kernel_size=3,input_shape=(retrain_input.shape[1],retrain_input.shape[2]),activation='relu',strides=1,padding='same'))
model.add(MaxPooling1D(pool_size=2,strides=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(300, activation='relu', kernel_regularizer=None, bias_regularizer=None,name='4'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid', kernel_regularizer=None, bias_regularizer=None,name='6'))
adam = keras.optimizers.Adam(lr=1e-4)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(retrain_input, data_y, epochs=300, batch_size=128, validation_split=0.1,callbacks=[EarlyStopping(patience=10), ModelCheckpoint(filepath='./conv_lstm_retrain.h5', save_best_only=True)],
                        shuffle=True, verbose=2)

/home/jiay/.local/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/300
10/10 - 1s - loss: 0.7264 - accuracy: 0.4790 - val_loss: 0.6922 - val_accuracy: 0.5000
Epoch 2/300
10/10 - 0s - loss: 0.6943 - accuracy: 0.5170 - val_loss: 0.6889 - val_accuracy: 0.5217
Epoch 3/300
10/10 - 0s - loss: 0.6913 - accuracy: 0.5347 - val_loss: 0.6871 - val_accuracy: 0.5290
Epoch 4/300
10/10 - 0s - loss: 0.6916 - accuracy: 0.5565 - val_loss: 0.6860 - val_accuracy: 0.5217
Epoch 5/300
10/10 - 0s - loss: 0.6763 - accuracy: 0.5670 - val_loss: 0.6921 - val_accuracy: 0.5217
Epoch 6/300
10/10 - 0s - loss: 0.6785 - accuracy: 0.5630 - val_loss: 0.6891 - val_accuracy: 0.5290
Epoch 7/300
10/10 - 0s - loss: 0.6662 - accuracy: 0.5800 - val_loss: 0.6874 - val_accuracy: 0.5290
Epoch 8/300
10/10 - 0s - loss: 0.6644 - accuracy: 0.6090 - val_loss: 0.6904 - val_accuracy: 0.5072
Epoch 9/300
10/10 - 0s - loss: 0.6566 - accuracy: 0.6204 - val_loss: 0.6897 - val_accuracy: 0.5362
Epoch 10/300
10/10 - 0s - loss: 0.6496 - accuracy: 0.6123 - val_loss: 0.6930 - val_accuracy: 0.5507
Epoch 11/